# Student's t-test for paired samples in Python:

In [1]:
import numpy as np
import sys
import timeit
import subprocess

from numba import jit

data1 = np.array([70.0, 71.0, 72.0, 73.0, 70.5])
data2 = np.array([70.15, 71.05, 71.95, 72.85, 70.45])

Functions definition:

In [2]:
@jit(nopython=True)
def avevar(data):
    # Given an array "data" of length "n", returns
    # its mean "ave" and its variance "var"
    n = len(data)
    ave, var = 0, 0
    
    # Mean
    for i in range(n):
        ave =  ave + data[i]
        
    ave = ave / n
    
    # Variance (of a sample population)
    for i in range(len(data)):
        s = data[i] - ave
        var =  var + s**2
        
    var = var / (n - 1)
    
    return ave, var

In [3]:
@jit(nopython=True)
def gammaln(xx):
    # Returns the value of ln[gamma(XX)] for X > 0. 
    # Full accuracy is obtained for XX > 1. 
    cof = np.array([76.18009173, -86.50532033, 24.01409822, 
                    -1.231739516, 0.120858003e-2, -0.536382e-5])
    stp = 2.50662827465
    
    x = xx - 1.0
    tmp = x + 5.5
    tmp = (x + 0.5) * np.log(tmp) - tmp
    ser = 1.0

    for i in range(len(cof)):
        x = x + 1
        ser = ser + cof[i] / (x)

    gammaln = tmp + np.log(stp * ser)
    return gammaln

In [4]:
def betacf(a, b, x):
    # Continued fraction for incomplete beta function, 
    # used by "betai"  
    eps = 3.0e-7
    imax = 100

    am = 1
    bm = 1
    az = 1
    qab = a + b
    qap = a + 1
    qam = a - 1
    bz = 1. - (qab * x) / qap

    # continued fraction evaluation using recurrence
    for m in range(1, imax):
        em = m
        tem = em + em
        d = (em * (b - m) * x) / ((qam + tem) * (a + tem))
        ap = az + d * am
        bp = bz + d * bm
        d = -((a + em) * (qab + em) * x) / ((a + tem) * (qap + tem))
        app = ap + d * az
        bpp = bp + d * bz
        aold = az
        am = ap / bpp
        bm = bp / bpp
        az = app / bpp
        bz = 1.0

        if (abs(az - aold) < eps * abs(az)):
            # print('BETACF: convergence OK')
            break

        if (m == imax):
            print('BETACF: a or b too big, or imax too small')
            sys.exit()

        betacf = az
        return betacf

In [5]:
def betai(a, b, x):
    # Returns the incomplete beta function Ix(a, b)

    if ((x < 0 ) or (x > 1)):
        print('BETAI: bad argument fo x')
        sys.exit()

    # Factor "bt" in front of continued fraction
    if ((x == 0 ) or (x == 1)):
        bt = 0
    else:
        bt = np.exp(gammaln(a + b) - gammaln(a) - gammaln(b) + \
             a * np.log(x) + b * np.log(1 - x))

    # Continued fraction
    if (x < ((a + 1) / (a + b + 2))):
        # ...directly
        betai = bt * betacf(a, b, x) / a
        return betai
    else:
        # ...after symmetry transformation
        betai = 1 - bt * betacf(b, a, 1 - x) / b
        return betai

In [6]:
def tptest(data1, data2):
    # Given the arrays "data1" and "data2" of length "n", returns
    # the Student's "t" for paired data and its significance as "prob".

    # Small values of "prob" indicates that the arrays have different
    # means.

    n1 = len(data1)
    n2 = len(data2)
  
    if (n1 != n2):
        print("Data arrays must have the same size")
    else:
        n = n1

    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    cov = 0.0
    for i in range(n):
        cov = cov + (data1[i] - ave1) * (data2[i] - ave2)

    df = n - 1.0
    cov = cov / df

    # Student's t
    t = (ave1 - ave2) / np.sqrt((var1 + var2 - 2.0 * cov) / n)

    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t**2))
    
    return t, prob

### Calling the ttest function in Python:

In [7]:
t, prob = tptest(data1, data2)

print(t, prob)

0.1961161351382784 0.8540797033791303


## Timing against Fortran:

Test function:

In [8]:
def test():
    for i in range(1000):
        tptest(data1, data2)

#### Python:

In [9]:
%timeit test()

24 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Fortran:

In [10]:
%timeit subprocess.call(['./fmain_tptest'])

7 ms ± 591 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
